In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
import re
import pandas as pd
import numpy as np
from glob import glob, iglob
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

Python 3.8.10 (default, May 26 2023, 14:05:08) 
[GCC 9.4.0] on linux


In [2]:
import utils
import gpt

LIBRARY = 'gs'
CATEGORY = 'jppy'
DATASET_FOLDER = f'./dataset/{LIBRARY}/{CATEGORY}/'
OUTPUT_FOLDER = './output/'

In [3]:
df_meta = pd.read_parquet(OUTPUT_FOLDER + f'meta-{LIBRARY}-{CATEGORY}.parquet')
print(df_meta.shape)

(73, 9)


In [4]:
df_meta.head(2)

,library,category,file_name,file_path,external_link,chapter,text_length,token_count,content
idx,,,,,,,,,
87c5c645171300baa4b6df99bb5ef28fa04fa265b666349e2c1375b7962a44ef,gs,jppy,Payroll Basics,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,1,1467,271,# Payroll Basics\nPurpose\nThe Payroll is base...
ebd0e02c76753f2896df4795a0155a6e0afab2d339a73924dfa7ee03b3a759bf,gs,jppy,Payroll Driver,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,2,993,210,# Payroll Driver\nDefinition\nProgram you use ...


In [5]:
df_sorted = df_meta.sort_values(by=['file_path', 'chapter'], ascending=True)
print(df_sorted.shape)

(73, 9)


In [6]:
df_sorted.sample(10)

,library,category,file_name,file_path,external_link,chapter,text_length,token_count,content
idx,,,,,,,,,
16033813e4a8496e1d2816add1564474d72c772f3c3d6025477ddf8458ab0ca1,gs,jppy,Transfer (Lost/Acquisition of Status),./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,50,644,143,# Transfer (Lost/Acquisition of Status)\nPurpo...
c9250b4509911d01929c2629816a987009a0904a853c2746c0707629b6385ca2,gs,jppy,"Administration of Health Insurance, Employee's...",./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,45,905,167,"# Administration of Health Insurance, Employee..."
ddc364e59695e80e3d12f81715dfb7c34348024a9030e358f2c50a8acb00a1c3,gs,jppy,Nursing Insurance Status Check,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,68,8601,1855,# Nursing Insurance Status Check\nUse\nThe fun...
7918a37865bcf619456ec368fe915becf10ed409fcfd5ef74d395088f1b171ce,gs,jppy,Acquisition of Qualification,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,46,1094,273,# Acquisition of Qualification\nPurpose\nThis ...
fe32709c55435dc70032d4c0c72d57231393161292ebefcb2be217824af2475c,gs,jppy,Infotypes for Payroll Japan,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,22,30,8,# Infotypes for Payroll Japan\n
0e8e1d119078b1ef56a8297e9578a5fdffbe0ac0f4c08efca9d4f6241e7f5e47,gs,jppy,Payroll Control,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,6,149,37,# Payroll Control\nUse\nThis function is used ...
69dd77a39836c678f1dee6fcf5eb9aecc8c3d35d2e8c176bffc438351bbe14e4,gs,jppy,Registrating in Infotype when Lost,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,49,907,231,# Registrating in Infotype when Lost\nUse\nThi...
ef5c26ff133d777269848c561a1a112ec52ee647a7e8ed120ae9b6381ab27658,gs,jppy,Commuter Allowance Information (Infotype 0559),./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,26,2702,546,# Commuter Allowance Information (Infotype 055...
6e9df30d981b0f04a01818e111f40523075dba8916fbe6ff95af128f5f6cf6e9,gs,jppy,Life and Injury Insurance Information Infotype...,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,29,2047,406,# Life and Injury Insurance Information Infoty...


In [7]:
idxs = []
embeddings = []
files = []
# count = 1
embedder = gpt.Embedding()
for idx, item in tqdm(df_meta.iterrows()):
#     print(idx, item.file)
    plain_text = item.content
    embedding = embedder.run(plain_text).result_
    
#     print(embedding)
    idxs.append(idx)
#     files.append(item.file_path)
    embeddings.append(embedding)
    
#     if count % 2000 == 0:
#         df_embedding = pd.DataFrame(np.array(embeddings))
#         df_embedding['idx'] = idxs
# #         df_embedding['file'] = files
#         df_embedding = df_embedding.set_index('idx')
#         print('exporting:', count, df_embedding.shape)
#         df_embedding.to_csv(f'./output/embedding-{CATEGORY}.csv', encoding='utf-8')
        
#     count = count + 1

73it [00:19,  3.68it/s]


Ada 0.0004 / 1000 tokens

We have 2317 chunks

2317 / 1000 * 0.0004 = 0.0008

<!-- 4.2 it/s, 38417 / 4.2 / 60 / 60 = 2.54hr -->

In [8]:
df_embedding = pd.DataFrame(np.array(embeddings))
df_embedding['idx'] = idxs
# df_embedding['file'] = files
df_embedding = df_embedding.set_index('idx')
print(df_embedding.shape)
# print('exporting:', count, df_embedding.shape)

(73, 1536)


In [9]:
df_embedding.head(2)

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
idx,,,,,,,,,,,,,,,,,,,,,
87c5c645171300baa4b6df99bb5ef28fa04fa265b666349e2c1375b7962a44ef,0.007614,0.010770,-0.002415,-0.044997,-0.052483,0.004925,-0.029431,-0.003909,-0.029701,-0.012033,...,0.031863,0.018120,0.016080,-0.042322,-0.029431,-0.039106,0.012155,0.002868,0.001954,-0.005185
ebd0e02c76753f2896df4795a0155a6e0afab2d339a73924dfa7ee03b3a759bf,0.011897,-0.002452,0.015097,-0.055882,-0.018056,0.007867,-0.016729,-0.016521,-0.011226,-0.008033,...,0.016743,0.005454,0.000401,-0.019038,-0.036057,-0.021181,-0.002336,0.010203,-0.007687,-0.012892


In [10]:
df_embedding.to_parquet(f'./output/embedding-{LIBRARY}-{CATEGORY}.parquet', index='idx')